In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
# import datetime
from datetime import datetime
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install pmdarima --quiet
import pmdarima as pm


ERROR: Could not find a version that satisfies the requirement pmdarima (from versions: none)
ERROR: No matching distribution found for pmdarima


ModuleNotFoundError: No module named 'pmdarima'

In [3]:

!pip install tvdatafeed --quiet
from tvDatafeed import TvDatafeed ,Interval

In [4]:
tv = TvDatafeed()
data = tv.get_hist(symbol='TCS',exchange='NSE',n_bars=5000)
data['date'] = data.index.astype(str)
new = data['date'].str.split(' ',expand=True)
data['date'] = new[0]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data

you are using nologin method, data you access may be limited


WebSocketAddressException: [Errno 11001] getaddrinfo failed

In [ ]:
timeseriesdf = data[['close']]
timeseriesdf

In [ ]:
timeseriessq = data['close']
timeseriessq

In [ ]:
data.info()

In [ ]:
data.index

# Visualisation

In [ ]:
plt.figure(figsize=(20,8))
fig = cf.Figure(data=[cf.Candlestick(x=data.index, 
                open=data['open'],
                high = data['high'],
                low = data['low'],
                close = data['close'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(data.close)

#### Histogram and Density Plots

In [ ]:
# create a histogram plot
plt.rc("figure", figsize=(20,20))
data.hist()

#### Box and Whisker Plots by Interval

In [ ]:
# create a boxplot of yearly data
databoxplot = data
databoxplot['date'] = databoxplot.index.astype(str)
new = databoxplot['date'].str.split(' ',expand=True)
databoxplot['date'] = new[0]
databoxplot = databoxplot[['date','close']]
databoxplot['year'] = databoxplot['date'].str[:4].astype(int)
databoxplot['date1'] = databoxplot['date'].str[5:]
databoxplot

In [ ]:
databoxplot.info()

In [ ]:
years = pd.pivot_table(databoxplot,index = 'date1', values = 'close',columns='year', aggfunc = sum)

years

In [ ]:
plt.figure(figsize = (20,8))
years.boxplot()

#### Lag plot

In [ ]:
# create a scatter plot
plt.figure(figsize = (20,8))
pd.plotting.lag_plot(timeseriesdf)

In [ ]:
# create an autocorrelation plot
from statsmodels.graphics.tsaplots import plot_acf

plt.rc("figure", figsize=(20,8))
plot_acf(timeseriesdf, lags=4000)
plt.show()

# Sampling and Transformation

In [ ]:
timeseriessq

#### Upsampling Data

In [ ]:
upsampled = timeseriessq.resample('H').mean()
upsampled.head(25)

In [ ]:
upsampled.shape

#### Interpolate the missing value

In [ ]:
interpolated = upsampled.interpolate(method='linear')
interpolated.head(25)

In [ ]:
plt.rc("figure", figsize=(20,8))
interpolated.plot()

#### Downsampling Data

In [ ]:
# downsample to quarterly intervals
resample = timeseriessq.resample('Q')
downsampled = resample.mean()

In [ ]:
downsampled.head()

In [ ]:
plt.rc("figure", figsize=(20,8))
downsampled.plot()

#### Tranformations

In [ ]:
# load and plot a time series
timeseriesdf

In [ ]:
# line plot
plt.subplot(221)
plt.plot(timeseriesdf)

# histogram
plt.subplot(222)
plt.hist(timeseriesdf)

plt.show()

#### Square Root Transform

In [ ]:
dataframe = pd.DataFrame(np.sqrt(timeseriesdf.values), columns = ['close'])
dataframe

In [ ]:
# line plot
plt.subplot(221)
plt.plot(dataframe['close'])

# histogram
plt.subplot(222)
plt.hist(dataframe['close'])

plt.show()

#### Log Transform

In [ ]:
dataframe = pd.DataFrame(np.log(timeseriesdf.values), columns = ['close'])
dataframe

In [ ]:
# line plot
plt.subplot(221)
plt.plot(dataframe['close'])

# histogram
plt.subplot(222)
plt.hist(dataframe['close'])
plt.show()

# Forecasting - Model Based

In [ ]:
heatmapdata = data[['date','close']]
heatmapdata['date'] = pd.to_datetime(heatmapdata['date'])
heatmapdata

In [ ]:
# Extracting Day, weekday name, month name, year from the Date column using 
# Date functions from pandas 

heatmapdata["month"] = heatmapdata['date'].dt.strftime("%b") # month extraction
heatmapdata["year"] = heatmapdata['date'].dt.strftime("%Y") # year extraction
heatmapdata["Day"] = heatmapdata['date'].dt.strftime("%d") # Day extraction
heatmapdata["wkday"] = heatmapdata['date'].dt.strftime("%A") # weekday extraction

heatmapdata

In [ ]:
heatmap_y_month = pd.pivot_table(data = heatmapdata,
                                 values = "close",
                                 index = "year",
                                 columns = "month",
                                 aggfunc = "mean",
                                 fill_value=0)
heatmap_y_month1 = heatmap_y_month[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]
heatmap_y_month1

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(heatmap_y_month1,
            annot=True,
            fmt="g",
            cmap = 'icefire') 

In [ ]:
# Boxplot for every
plt.figure(figsize=(20,10))

plt.subplot(221)
sns.boxplot(x="month",y="close",data=heatmapdata)

plt.subplot(222)
sns.boxplot(x="year",y="close",data=heatmapdata)

In [ ]:
sns.lineplot(x="year",y="close",data=heatmapdata)

#### Splitting data

In [ ]:
data1 = heatmapdata
data1

In [ ]:
data1['t'] = np.arange(1,data1.shape[0]+1)
data1['t_square'] = np.square(data1.t)
data1['log_close'] = np.log(data1.close)
data2 = pd.get_dummies(data1['month'])
data1 = pd.concat([data1, data2],axis=1)
data1 = data1.reset_index(drop = True)
data1

In [ ]:
# Using 3/4th data for training and remaining for testing
test_size = round(0.25 * (data1.shape[0]+1))

Train = data1[:-test_size]
Test = data1[-test_size:]

In [ ]:
Train

In [ ]:
Test

### Trying basic models

In [ ]:
#Linear Model
import statsmodels.formula.api as smf 

linear_model = smf.ols('close~t',data=Train).fit()
pred_linear =  pd.Series(linear_model.predict(pd.DataFrame(Test['t'])))
rmse_linear = np.sqrt(np.mean((np.array(Test['close'])-np.array(pred_linear))**2))
rmse_linear

In [ ]:
#Exponential
Exp = smf.ols('log_close~t',data=Train).fit()
pred_Exp = pd.Series(Exp.predict(pd.DataFrame(Test['t'])))
rmse_Exp = np.sqrt(np.mean((np.array(Test['close'])-np.array(np.exp(pred_Exp)))**2))
rmse_Exp

In [ ]:
#Quadratic 
Quad = smf.ols('close~t+t_square',data=Train).fit()
pred_Quad = pd.Series(Quad.predict(Test[["t","t_square"]]))
rmse_Quad = np.sqrt(np.mean((np.array(Test['close'])-np.array(pred_Quad))**2))
rmse_Quad

In [ ]:
#Additive seasonality 
add_sea = smf.ols('close~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=Train).fit()
pred_add_sea = pd.Series(add_sea.predict(Test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']]))
rmse_add_sea = np.sqrt(np.mean((np.array(Test['close'])-np.array(pred_add_sea))**2))
rmse_add_sea

In [ ]:
#Additive Seasonality Quadratic 
add_sea_Quad = smf.ols('close~t+t_square+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=Train).fit()
pred_add_sea_quad = pd.Series(add_sea_Quad.predict(Test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','t','t_square']]))
rmse_add_sea_quad = np.sqrt(np.mean((np.array(Test['close'])-np.array(pred_add_sea_quad))**2))
rmse_add_sea_quad

In [ ]:
##Multiplicative Seasonality
Mul_sea = smf.ols('log_close~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = Train).fit()
pred_Mult_sea = pd.Series(Mul_sea.predict(Test))
rmse_Mult_sea = np.sqrt(np.mean((np.array(Test['close'])-np.array(np.exp(pred_Mult_sea)))**2))
rmse_Mult_sea

In [ ]:
#Multiplicative Additive Seasonality 
Mul_Add_sea = smf.ols('log_close~t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = Train).fit()
pred_Mult_add_sea = pd.Series(Mul_Add_sea.predict(Test))
rmse_Mult_add_sea = np.sqrt(np.mean((np.array(Test['close'])-np.array(np.exp(pred_Mult_add_sea)))**2))
rmse_Mult_add_sea 

In [ ]:
#Compare the results 
datamodel = {"MODEL":pd.Series(["rmse_linear","rmse_Exp","rmse_Quad","rmse_add_sea","rmse_add_sea_quad","rmse_Mult_sea","rmse_Mult_add_sea"]),"RMSE_Values":pd.Series([rmse_linear,rmse_Exp,rmse_Quad,rmse_add_sea,rmse_add_sea_quad,rmse_Mult_sea,rmse_Mult_add_sea])}
table_rmse=pd.DataFrame(datamodel)
table = table_rmse.sort_values(['RMSE_Values'],ignore_index = True)
table

In [ ]:
bestmodel = table.iloc[0,0]
bestmodel

if bestmodel == "rmse_linear" :
  formula = 'close~t'

if bestmodel == "rmse_Exp":
  formula = 'log_close~t'

if bestmodel == "rmse_Quad" :
  formula = 'close~t+t_square'

if bestmodel == "rmse_add_sea":
  formula = 'close~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov'

if bestmodel == "rmse_add_sea_quad":
  formula = 'close~t+t_square+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov'

if bestmodel == "rmse_Mult_sea":
  formula = 'log_close~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov'

if bestmodel == "rmse_Mult_add_sea":
  formula = 'log_close~t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov'

formula

In [ ]:
#Build the model on entire data set
model_full = smf.ols(formula,data=data1).fit()

In [ ]:
pred_new  = pd.Series(model_full.predict(data1))
pred_new

In [ ]:
if bestmodel == "rmse_Exp" or "rmse_Mult_sea" or "rmse_Mult_add_sea":
  data1["forecasted_close"] = pd.Series(np.exp(pred_new))
else:
  data1["forecasted_close"] = pd.Series((pred_new))

In [ ]:
plt.figure(figsize = (20,8))

plt.plot(data1[['close','forecasted_close']].reset_index(drop=True))

In [ ]:
data1

# Forecasting - Data Driven

In [ ]:
import statsmodels.graphics.tsaplots as tsa_plots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing # SES
from statsmodels.tsa.holtwinters import Holt # Holts Exponential Smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing 


#### Moving Average 

In [ ]:
x=20

plt.figure(figsize=(24,7))
data1['close'].plot(label="org")
data1["close"].rolling(x).mean().plot(label=str(x))
plt.legend(loc='best')

In [ ]:
plt.figure(figsize=(24,7))
data1['close'].plot(label="org")
for i in range(50,201,50):
    data1["close"].rolling(i).mean().plot(label=str(i))
plt.legend(loc='best')

#### Time series decomposition plot 


In [ ]:
decompose_ts_add = seasonal_decompose(data1['close'], period = 365)
decompose_ts_add.plot()

plt.show()

#### ACF plots and PACF plots


In [ ]:
tsa_plots.plot_acf(data1.close,lags=350)
tsa_plots.plot_pacf(data1.close,lags=350)
plt.show()

### Evaluation Metric RMSE

In [ ]:
def RMSE(pred,org):
  MSE = np.square(np.subtract(org,pred)).mean()   
  return np.sqrt(MSE) 

### Simple Exponential Method


In [ ]:
ses_model = SimpleExpSmoothing(Train["close"]).fit(smoothing_level=0.2)
pred_ses = ses_model.predict(start = Test.index[0],end = Test.index[-1])
RMSE(pred_ses,Test.close)

### Holt method 

In [ ]:
# Holt method 
hw_model = Holt(Train["close"]).fit(smoothing_level=0.8, smoothing_slope=0.2)
pred_hw = hw_model.predict(start = Test.index[0],end = Test.index[-1])
RMSE(pred_hw,Test.close) 

### Holts winter exponential smoothing with additive seasonality and additive trend


In [ ]:
hwe_model_add_add = ExponentialSmoothing(Train["close"],seasonal="add",trend="add",seasonal_periods=365).fit() #add the trend to the model
pred_hwe_add_add = hwe_model_add_add.predict(start = Test.index[0],end = Test.index[-1])
RMSE(pred_hwe_add_add,Test.close) 

### Holts winter exponential smoothing with multiplicative seasonality and additive trend

In [ ]:
hwe_model_mul_add = ExponentialSmoothing(Train["close"],seasonal="mul",trend="add",seasonal_periods=365).fit() 
pred_hwe_mul_add = hwe_model_mul_add.predict(start = Test.index[0],end = Test.index[-1])
RMSE(pred_hwe_mul_add,Test.close)

### Final Model by combining train and test

In [ ]:
hwe_model_mul_add = ExponentialSmoothing(data["close"],seasonal="mul",trend="add",seasonal_periods=365).fit()

In [ ]:
#Forecasting for next 12 time periods
forecasted = hwe_model_mul_add.forecast(730)

In [ ]:
plt.figure(figsize=(24,7))
plt.plot(data1.close, label = "Actual")
plt.plot(forecasted, label = "Forecasted")
plt.legend()

# Forecasting using ARIMA model

In [ ]:
# Import libraries
import itertools
from math import sqrt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# Perform Augmented Dickey-Fuller unit root test
result = adfuller(data1['close'])

print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))
print('Critical Values:')
for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))

In [ ]:
# P-Value is higher than 0.05
# Check if series is stationary

def get_stationarity(timeseriessq):
    
    # rolling statistics
    rolling_mean = timeseriessq.rolling(window=365).mean()
    rolling_std = timeseriessq.rolling(window=365).std()
    
    # rolling statistics plot
    original = plt.plot(timeseriessq, color='blue', label='Original')
    mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
    std = plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    # Dickey–Fuller test:
    result = adfuller(timeseriessq)
    print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

In [ ]:
rolling_mean = data1['log_close'].rolling(window=365).mean()
data1_log_minus_mean = data1['log_close'] - rolling_mean
data1_log_minus_mean.dropna(inplace=True)

get_stationarity(data1_log_minus_mean)

In [ ]:
rolling_mean_exp_decay = data1['log_close'].ewm(halflife=365, min_periods=0, adjust=True).mean()
df_log_exp_decay = data1['log_close'] - rolling_mean_exp_decay
df_log_exp_decay.dropna(inplace=True)

get_stationarity(df_log_exp_decay)

In [ ]:
# Above tests show that the timeseries is not stationary.
# still performing ARIMA

#### Persistence/ Base model

In [ ]:
# evaluate a persistence model
print('Dataset %d, Validation %d' % (len(Train), len(Test)))

In [ ]:
# prepare data
train = Train.close.values.astype('float32')
train

In [ ]:
test = Test.close.values.astype('float32')
test

In [ ]:
# walk-forward validation
history = [x for x in train]
history

#### Naive Predictions

In [ ]:
predictions = list()

for i in range(len(test)):
    yhat = history[-1]
    predictions.append(yhat)
    
    # observation
    obs = test[i]
    history.append(obs)
    
    print('>Predicted=%.3f, Expected=%.3f' % (yhat, obs))

In [ ]:
# report performance
rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

# Auto ARIMA

In [ ]:
# load dataset
train = Train.close.squeeze()
train

In [ ]:
#Standard ARIMA Model
ARIMA_model = pm.auto_arima(data1['close'], 
                      start_p=1, 
                      start_q=1,
                      test='adf', # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1, # frequency of series (if m==1, seasonal is set to FALSE automatically)
                      d=None,# let model determine 'd'
                      seasonal=False, # No Seasonality for standard ARIMA
                      trace=False, #logs 
                      error_action='warn', #shows errors ('ignore' silences these)
                      suppress_warnings=True,
                      stepwise=True)

In [ ]:
ARIMA_model.plot_diagnostics(figsize=(20,12))
plt.show()

In [ ]:
from pandas.tseries.frequencies import DAYS
def forecast(ARIMA_model, periods=730):
    # Forecast
    n_periods = periods
    fitted, confint = ARIMA_model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = pd.date_range(data.index[-1] + pd.DateOffset(days=1), periods = n_periods, freq='D')
    
    # make series for plotting purpose
    fitted_series = pd.Series(fitted.values, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    # Plot
    plt.figure(figsize=(15,7))
    plt.plot(data["close"])
    plt.plot(fitted_series, color='darkgreen')
    plt.fill_between(lower_series.index, 
                    lower_series, 
                    upper_series, 
                    color='k', alpha=.15)

    plt.title("ARIMA - Forecast of Close Price")
    plt.show()

forecast(ARIMA_model)

#### Build Model based on the optimized values

In [ ]:
from statsmodels.graphics.tsaplots import plot_predict
plot_predict(ARIMA_model,1,96)

#SARIMA

#SARIMAX

#VAR

#VARMA

#VARMAX

#LSTM ANN

#FB PROPHET